In [1]:
# importing and configering libs

In [2]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import cross_validate,cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn import metrics
from sklearn.metrics import r2_score

pd.set_option('display.max_columns',2000) # show all columns in jupyter
pd.set_option('display.max_rows', 100) # display 100 at least rows

In [3]:
# Loading the data

In [4]:
colsToLoad =["INSTNM","ADM_RATE","ADM_RATE_ALL","ACTCMMID","ACTENMID","ACTMTMID","ACTWRMID","SAT_AVG","SAT_AVG_ALL","SATMTMID","UGDS","HIGHDEG", "CONTROL", "TUITFTE", "INEXPFTE", "AVGFACSAL", "COSTT4_P",
             "COSTT4_A", "PCTFLOAN", "PCTPELL", "COMP_ORIG_YR2_RT","UGDS_WHITE","UGDS_BLACK","UGDS_HISP","UGDS_ASIAN","UGDS_AIAN","UGDS_NHPI","UGDS_2MOR","UGDS_NRA","UGDS_UNKN","PPTUG_EF","COSTT4_A","TUITIONFEE_IN","TUITIONFEE_OUT","TUITIONFEE_PROG","TUITFTE","AVGFACSAL","DEATH_YR3_RT","COMP_ORIG_YR3_RT","LOAN_DEATH_YR3_RT","LOAN_COMP_ORIG_YR3_RT","DEATH_YR4_RT","COMP_ORIG_YR4_RT","COMPL_RPY_1YR_RT","NONCOM_RPY_1YR_RT","AGE_ENTRY","COUNT_NWNE_P10","COUNT_WNE_P10","MN_EARN_WNE_P10","MD_EARN_WNE_P10","COMPL_RPY_1YR_RT","NONCOM_RPY_1YR_RT","COMPL_RPY_3YR_RT","NONCOM_RPY_3YR_RT","COMPL_RPY_5YR_RT","NONCOM_RPY_5YR_RT","COMPL_RPY_7YR_RT","NONCOM_RPY_7YR_RT"]

# As each data row represent a specific institute , So we can use institute name as an index
index_feature = "INSTNM"

scoreCardDF = pd.read_csv("I:\\NU Project\\Git-Repo\\Data\\MERGED2013_14_PP.csv", index_col=index_feature, usecols=colsToLoad)
# ignore "MEDIAN_HH_INC" column, as 0 non-null float64

F:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (1606) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# General exploration of data 

In [6]:
scoreCardDF.head()

,HIGHDEG,CONTROL,ADM_RATE,ADM_RATE_ALL,SATMTMID,ACTCMMID,ACTENMID,ACTMTMID,ACTWRMID,SAT_AVG,SAT_AVG_ALL,UGDS,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,COSTT4_A,COSTT4_P,TUITIONFEE_IN,TUITIONFEE_OUT,TUITIONFEE_PROG,TUITFTE,INEXPFTE,AVGFACSAL,PCTPELL,PCTFLOAN,COMP_ORIG_YR2_RT,DEATH_YR3_RT,COMP_ORIG_YR3_RT,LOAN_DEATH_YR3_RT,LOAN_COMP_ORIG_YR3_RT,DEATH_YR4_RT,COMP_ORIG_YR4_RT,COMPL_RPY_1YR_RT,NONCOM_RPY_1YR_RT,COMPL_RPY_3YR_RT,NONCOM_RPY_3YR_RT,COMPL_RPY_5YR_RT,NONCOM_RPY_5YR_RT,COMPL_RPY_7YR_RT,NONCOM_RPY_7YR_RT,AGE_ENTRY,COUNT_NWNE_P10,COUNT_WNE_P10,MN_EARN_WNE_P10,MD_EARN_WNE_P10
INSTNM,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Alabama A & M University,4,1,0.8989,0.8989,400.0,17.0,17.0,17.0,NaN,823.0,823.0,4051.0,0.0279,0.9501,0.0089,0.0022,0.0012,0.0010,0.0000,0.0002,0.0084,0.0622,18888.0,NaN,7182.0,12774.0,NaN,9063.0,7459.0,7079.0,0.7115,0.8204,0.034958602,PrivacySuppressed,0.097451274,PrivacySuppressed,0.087332809,PrivacySuppressed,0.241845664,0.3075801749,0.1213692946,0.388185654,0.1768901569,0.5305514158,0.2299150882,0.7553324969,0.4143646409,20.540410133,246,1741,34300,29900
University of Alabama at Birmingham,4,1,0.8673,0.8673,585.0,25.0,26.0,23.0,NaN,1146.0,1146.0,11200.0,0.5987,0.2590,0.0258,0.0518,0.0026,0.0007,0.0344,0.0140,0.0130,0.2579,19990.0,NaN,7206.0,16398.0,NaN,9033.0,17208.0,10170.0,0.3505,0.5397,0.150529204,PrivacySuppressed,0.333945628,PrivacySuppressed,0.332664527,PrivacySuppressed,0.422933112,0.578436134,0.3365384615,0.6351660939,0.4240231548,0.7312626604,0.5116761806,0.8288419519,0.6054718477,23.45881476,470,3474,46400,40200
Amridge University,4,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,322.0,0.2919,0.4224,0.0093,0.0031,0.0031,0.0031,0.0000,0.0000,0.2671,0.3727,12300.0,NaN,6870.0,6870.0,NaN,12057.0,5123.0,3849.0,0.6839,0.7629,0.058823529,PrivacySuppressed,0.161849711,PrivacySuppressed,PrivacySuppressed,PrivacySuppressed,0.209580838,0.3859649123,0.1542699725,0.6226415094,0.3043478261,0.5192307692,0.4144144144,PrivacySuppressed,PrivacySuppressed,33.925581395,57,168,46100,40100
University of Alabama in Huntsville,4,1,0.8062,0.8062,580.0,26.0,26.0,25.0,NaN,1180.0,1180.0,5525.0,0.7012,0.1310,0.0338,0.0364,0.0145,0.0002,0.0161,0.0329,0.0338,0.2395,20306.0,NaN,9192.0,21506.0,NaN,8322.0,9352.0,9341.0,0.3281,0.4728,0.157939189,PrivacySuppressed,0.284602076,PrivacySuppressed,0.283109405,PrivacySuppressed,0.393046107,0.639589169,0.3486956522,0.7177142857,0.3818860878,0.7959731544,0.474012474,0.8810810811,0.6389140271,24.10127697,188,1361,50500,45600
Alabama State University,4,1,0.5125,0.5125,425.0,17.0,17.0,17.0,NaN,830.0,830.0,5354.0,0.0161,0.9285,0.0114,0.0015,0.0009,0.0007,0.0064,0.0207,0.0138,0.0902,17400.0,NaN,8720.0,15656.0,NaN,7813.0,7393.0,6557.0,0.8265,0.8735,0.028949025,PrivacySuppressed,0.115574348,PrivacySuppressed,0.10949464,PrivacySuppressed,0.187935035,0.2118863049,0.0985550204,0.3057553957,0.1694465504,0.4466019417,0.2263289037,0.7058823529,0.3645320197,20.648575305,285,1977,29500,26700


In [7]:
scoreCardDF.shape

(7804, 52)

In [8]:
scoreCardDF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7804 entries, Alabama A & M University to Georgia Military College-Stone Mountain
Data columns (total 52 columns):
HIGHDEG                  7804 non-null int64
CONTROL                  7804 non-null int64
ADM_RATE                 2221 non-null float64
ADM_RATE_ALL             2486 non-null float64
SATMTMID                 1317 non-null float64
ACTCMMID                 1344 non-null float64
ACTENMID                 1167 non-null float64
ACTMTMID                 1168 non-null float64
ACTWRMID                 300 non-null float64
SAT_AVG                  1422 non-null float64
SAT_AVG_ALL              1532 non-null float64
UGDS                     7091 non-null float64
UGDS_WHITE               7091 non-null float64
UGDS_BLACK               7091 non-null float64
UGDS_HISP                7091 non-null float64
UGDS_ASIAN               7091 non-null float64
UGDS_AIAN                7091 non-null float64
UGDS_NHPI                7091 non-null float64

In [9]:
# Assuming that the output or target variable is COMPL_RPY_1YR_RT
target_feature = "COMPL_RPY_1YR_RT"

In [10]:
# get the number of null values at each column to check if target variable COMPL_RPY_1YR_RT has null at any rows or not
scoreCardDF.isnull().sum()

HIGHDEG                     0
CONTROL                     0
ADM_RATE                 5583
ADM_RATE_ALL             5318
SATMTMID                 6487
ACTCMMID                 6460
ACTENMID                 6637
ACTMTMID                 6636
ACTWRMID                 7504
SAT_AVG                  6382
SAT_AVG_ALL              6272
UGDS                      713
UGDS_WHITE                713
UGDS_BLACK                713
UGDS_HISP                 713
UGDS_ASIAN                713
UGDS_AIAN                 713
UGDS_NHPI                 713
UGDS_2MOR                 713
UGDS_NRA                  713
UGDS_UNKN                 713
PPTUG_EF                  732
COSTT4_A                 3665
COSTT4_P                 5261
TUITIONFEE_IN            3298
TUITIONFEE_OUT           3517
TUITIONFEE_PROG          5056
TUITFTE                   442
INEXPFTE                  442
AVGFACSAL                3150
PCTPELL                   741
PCTFLOAN                  741
COMP_ORIG_YR2_RT          207
DEATH_YR3_

In [11]:
# Data preprocessing

In [12]:
# drop all rows that do not have a value for out target variable
scoreCardDF = scoreCardDF[(scoreCardDF[target_feature] != "PrivacySuppressed")\
                          & (scoreCardDF[target_feature] != np.nan)\
                          & (scoreCardDF[target_feature] != None)\
                          &(scoreCardDF[target_feature].notna())]
scoreCardDF.shape

(5632, 52)

In [13]:
# Extract input features by exluding the index feature and target output feature from the loadd features
input_features =  [f for f in colsToLoad if f not in [index_feature,target_feature]] 
# Specify categorial features columns
cat_columns = ["HIGHDEG","CONTROL"]
# Extract input float features by excluding categorial features
float_columns = [f for f in input_features if f not in cat_columns]

In [14]:
# As some columns have PrivacySuppressed as a value to indicate that the value is missing/hidden due to data privacy
# we define function that loop across all columns (features) to replace "PrivacySuppressed" value with null/nan 
# and convert it into float type
def cleanPrivacySuppressed(dataFrame):
    for colName in dataFrame.columns:
        dataFrame[colName] = dataFrame[colName].replace("PrivacySuppressed", np.nan)
        dataFrame[colName] = dataFrame[colName].astype(float)
    return dataFrame

# Build an estimator for this function to be used by ML pipeline
from sklearn.base import BaseEstimator, TransformerMixin
class PrivacySuppressedHandler(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self  
    def transform(self, X, y=None):
        return cleanPrivacySuppressed(X)

In [15]:
# pipeline for all float columns
# it is a separate pipline, because there is a potential to add more preprocessing steps based on model evaluation
float_pipeline = ColumnTransformer([("privacy_suppressed_handler", PrivacySuppressedHandler(),float_columns)])

In [16]:
# pipeline for all Categoricol (discrete) columns
from sklearn.preprocessing import OneHotEncoder
cat_pipeline = ColumnTransformer([("hot_encoder",OneHotEncoder(sparse=False),cat_columns)])

In [17]:
# Build a full pipeline for preprocessing by merge these two pipelines in one
from sklearn.pipeline import FeatureUnion
full_pipeline = FeatureUnion(transformer_list=[("float_pipeline", float_pipeline),("cat_pipeline", cat_pipeline)])

In [18]:
# Execute the preprocessing pipeline of the input features
processed_input_features = full_pipeline.fit_transform(scoreCardDF)
processed_input_features = pd.DataFrame(processed_input_features)
target_output = scoreCardDF[target_feature]

F:\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [19]:
# Split the data into training and test sets with ration 80 / 20
input_train, input_test, output_train, output_test =\
    train_test_split(processed_input_features, target_output.values, test_size=0.2, random_state=0)

In [20]:
# deine function to impute/replace null values with mean value across all columns of training datafram using scikit learn imputer
from sklearn.impute import SimpleImputer
def imputeNullWithMean(dataFrame):
    imputer = SimpleImputer(strategy='mean')
    imputed_data = pd.DataFrame(imputer.fit_transform(dataFrame), columns=dataFrame.columns)
    return imputed_data , imputer

In [21]:
# imputation using mean value
input_train ,imputer = imputeNullWithMean(input_train)
input_test = imputer.fit_transform(input_test)
#input_train.isnull().sum()

In [ ]:
# Training the algorithm (SVR - SVM for regression)
# svr_poly = SVR(kernel='poly', C=100, gamma='auto', degree=3, epsilon=.1, coef0=1)
svr_linear = SVR(kernel='linear') # , C=1, epsilon=0.2
svr_linear.fit(input_train, output_train)

In [ ]:
# To make predictions on the test data
output_pred = svr_linear.predict(input_test)

In [25]:
# To compare the actual output values for input_test which is (output_test) with the predicted values output_pred
df = pd.DataFrame({'Actual': output_test, 'Predicted': output_pred})
df

,Actual,Predicted
0,0.4418777943,0.504706
1,0.6991869919,0.504706
2,0.2754491018,0.504706
3,0.4281217209,0.504706
4,0.6611295681,0.504706
...,...,...
1122,0.575,0.504706
1123,0.5616024187,0.504706
1124,0.5319148936,0.504706
1125,0.3049327354,0.504706


In [26]:
# The final step is to evaluate the performance of algorithm. 
# We'll do this by finding the values for MAE, MSE, RMSE, and RSE
print('Mean Absolute Error:', metrics.mean_absolute_error(output_test, output_pred))
print('Mean Squared Error:', metrics.mean_squared_error(output_test, output_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(output_test, output_pred)))
print('Root Squared Error:', r2_score(output_test, output_pred))

Mean Absolute Error: 0.16959921472458375
Mean Squared Error: 0.0395792478237893
Root Mean Squared Error: 0.19894533878377071
Root Squared Error: -0.00360251105582976


In [27]:
# catHotColumns is how oneHotEncoder treats HIGHDEG AND CONTROL in preprocessing
catHotColumns =["HIGHDEG_Non-degree-granting","HIGHDEG_Certificate_degree","HIGHDEG_Associate_degree","HIGHDEG_Associate_degree","HIGHDEG_Associate_degree","CONTROL_Public","CONTROL_Private_nonprofit","CONTROL_Private for-profit"]
# ADD all toghether. this is the structre that gets out from the preprocessing pipline
all_features = float_columns.copy()
all_features.extend(catHotColumns)

In [29]:
# get the features importance
feature_name_importance = zip(all_features, svr_linear.coef_)
sorted(feature_name_importance, key=lambda x: x[1], reverse=True)

AttributeError: coef_ is only available when using a linear kernel

In [ ]:
#  in case of multivariable linear regression, the regression model has to find the most optimal coefficients for all the attributes. 
# To see what coefficients our regression model has chosen
#coeffDF = pd.DataFrame(svr_poly.coef_, scoreCardDF.columns, columns=['Coefficient'])
#coeffDF

In [23]:
# see coefficient and p-value
import statsmodels.api as sm
processed_input_feature.scolumns = all_features.columns
mod = sm.OLS(target_output.values,processed_input_features)
fii = mod.fit()
# p_values = fii.summary2().tables[1]['P>|t|']
print(fii.summary())

In [106]:
targetFeatureMean = scoreCardDF[target_feature].mean()
targetFeatureMean

TypeError: Could not convert 0.30758017490.5784361340.38596491230.6395891690.21188630490.6477024070.31372549020.51987110630.59348441930.74308755760.79761904760.260.39398889250.51405622490.35670103090.62134831460.38258685640.60294117650.54187544740.62979683970.57805907170.60.461059190.56179775280.18076923080.52316076290.55976095620.47342995170.65487674170.55203619910.35779816510.78685258960.56897477190.71238938050.4250.20786516850.17142857140.49436550710.5148936170.50.73472803350.69539078160.65972222220.640.69892473120.3150582050.73472803350.49010989010.22817460320.2621599340.29135538950.27190332330.39545719280.3525305410.3525305410.3525305410.08757062150.12432847280.17559899120.63618897910.58333333330.75506528590.30476190480.3453237410.47031963470.24088480640.45901639340.40779220780.40779220780.47136563880.35768261960.74872448980.56042105260.30590909090.35652173910.36307053940.41764705880.28294587920.28294587920.48571428570.42144224630.61484738750.58163265310.48979591840.4480.347721359